# Load Data from Google Drive

---

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!unzip 'drive/MyDrive/Colab Notebooks/uw-cs480-fall20.zip'

Streaming output truncated to the last 5000 lines.
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/3212.jpg  
  inflating: __MACOSX/uw-cs480-fall20/suffled-images/shuffled-images/._3212.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/11755.jpg  
  inflating: __MACOSX/uw-cs480-fall20/suffled-images/shuffled-images/._11755.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/52888.jpg  
  inflating: __MACOSX/uw-cs480-fall20/suffled-images/shuffled-images/._52888.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/12274.jpg  
  inflating: __MACOSX/uw-cs480-fall20/suffled-images/shuffled-images/._12274.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/44032.jpg  
  inflating: __MACOSX/uw-cs480-fall20/suffled-images/shuffled-images/._44032.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/14605.jpg  
  inflating: __MACOSX/uw-cs480-fall20/suffled-images/shuffled-images/._14605.jpg  
  inflating: uw-cs480

## Import Library

In [4]:
# libraries
import math
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
from torch.nn import Linear, ReLU, CrossEntropyLoss
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.image as img

# Using CNN to predict categories with images

## Process Data

In [6]:
def load_data():
    train_file = "uw-cs480-fall20/train.csv"
    test_file = "uw-cs480-fall20/test.csv"

    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)

    return df_train, df_test

In [7]:
df_train, df_test = load_data()

n_train_data = len(df_train)
n_test_data = len(df_test)
train_data = df_train[['id', 'category']]
test_data = df_test['id']

train_data.head()

,id,category
0,36274,Scarves
1,15129,Flip Flops
2,58976,Topwear
3,32922,Sandal
4,29561,Topwear


In [8]:
CATEGORY_ENCODER = LabelEncoder()
CATEGORY_ENCODER.fit(train_data['category'])
train_data['category'] = CATEGORY_ENCODER.transform(train_data['category'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
train_df, valid_df = train_test_split(train_data, test_size=0.2)

In [10]:
train_df.reset_index(drop=True, inplace=True)
valid_df.reset_index(drop=True, inplace=True)

In [20]:
def load_img(id, transform = None):
    path = 'uw-cs480-fall20/suffled-images/shuffled-images/' + str(id) +'.jpg'
    image = img.imread(path)
    image = transforms.ToTensor()(image)
    return image

From [pytorch tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html), process the image data

In [13]:
class ImageDataset(Dataset):
    def __init__(self, train_df):
        super().__init__()
        self.id = train_df['id']
        self.label = train_df['category']

    def __len__(self):
        return len(self.id)
    
    def __getitem__(self,index):
        label = self.label[index]
        id = self.id[index]
        image = load_img(id)
        
        return image, label

In [14]:
# Hyper parameters

num_epochs = 20
num_classes = 27
batch_size = 32
learning_rate = 0.001

In [15]:
train_dataset = ImageDataset(train_df)
train_iterator = DataLoader(train_dataset, batch_size=32, shuffle=True)

valid_dataset = ImageDataset(valid_df)
valid_iterator = DataLoader(valid_dataset, batch_size=32, shuffle=True)

# Create CNN model

Follow the [Guide](https://www.pluralsight.com/guides/image-classification-with-pytorch) to build CNN and related functions

In [16]:
class CNN(nn.Module): 
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(32*39*29, 1024)
        self.drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(1024, 27)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.act(self.conv1(x)) # [batch_size, 32, 78, 58]
        x = self.pool(x) # [batch_size, 32, 39, 29]
        x = x.view(x.size(0), -1) # [batch_size, 32*39*29]
        x = self.act(self.fc1(x)) # [batch_size, 128]
        x = self.drop(x)
        x = self.fc2(x) # [batch_size, 27]
        return x

# Related functions for CNN

In [22]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for images, labels in iterator:
        optimizer.zero_grad()
                
        y_pred = model(images)
        
        
        loss = criterion(y_pred, labels)
        
        acc = calculate_accuracy(y_pred, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate_cnn(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():

        for images, labels in iterator:

            y_pred = model(images)
            
            loss = criterion(y_pred, labels)
            
            acc = calculate_accuracy(y_pred, labels)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [18]:
criterion = CrossEntropyLoss()

model = CNN()

optimizer = Adam(model.parameters(), lr=0.001)

In [ ]:
train_accuracy_list = []
train_loss_list = []
valid_acc_list = []
valid_loss_list = []
best_valid_loss = float('inf')

for epoch in range(20):

    start_time = time.monotonic()

    for image, label in train_iterator:
      # train
      train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
      
      # valid
      valid_loss, valid_acc = evaluate_cnn(model, valid_iterator, criterion)

      # save best model
      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model, 'cnn-model.pt')

      # Track the accuracy
      train_accuracy_list.append(train_acc)
      train_loss_list.append(train_loss)
      valid_acc_list.append(valid_acc)
      valid_loss_list.append(valid_loss)
          
      # print epoch info
      end_time = time.monotonic()
      epoch_mins, epoch_secs = epoch_time(start_time, end_time)
      print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
      print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 4m 48s
	Train Loss: 0.718 | Train Acc: 80.47%
	 Val. Loss: 0.500 |  Val. Acc: 85.20%
Epoch: 01 | Epoch Time: 11m 19s
	Train Loss: 0.422 | Train Acc: 87.75%
	 Val. Loss: 0.478 |  Val. Acc: 87.23%
Epoch: 01 | Epoch Time: 17m 56s
	Train Loss: 0.306 | Train Acc: 91.19%
	 Val. Loss: 0.405 |  Val. Acc: 88.86%
Epoch: 01 | Epoch Time: 24m 34s
	Train Loss: 0.224 | Train Acc: 93.28%
	 Val. Loss: 0.361 |  Val. Acc: 90.05%
Epoch: 01 | Epoch Time: 31m 11s
	Train Loss: 0.166 | Train Acc: 94.80%
	 Val. Loss: 0.388 |  Val. Acc: 90.49%
Epoch: 01 | Epoch Time: 37m 44s
	Train Loss: 0.115 | Train Acc: 96.30%
	 Val. Loss: 0.390 |  Val. Acc: 90.25%
Epoch: 01 | Epoch Time: 44m 22s
	Train Loss: 0.084 | Train Acc: 97.30%
	 Val. Loss: 0.437 |  Val. Acc: 90.20%
Epoch: 01 | Epoch Time: 51m 0s
	Train Loss: 0.064 | Train Acc: 97.89%
	 Val. Loss: 0.549 |  Val. Acc: 89.36%


## Use keras

In [ ]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

In [5]:
train_category = df_train["category"]
all_categories = list(set(train_category))
n_categories = len(all_categories)
y_train = []

for i in range(n_train_data):
    current_category = train_category[i]
    index = all_categories.index(current_category)
    y_train.append(index)

In [6]:
# parameters for this script
batch_size = 32
num_classes = 27

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)

# normalize the data
x_train = x_train.astype('float32')
x_train /= 255
x_test = x_test.astype('float32')
x_test /= 255

In [4]:
from keras.preprocessing.image import img_to_array, load_img

train_img = []
test_img = []
for i in range(n_train_data):
    img = load_img('uw-cs480-fall20/suffled-images/shuffled-images/' + str(train_id[i]) +'.jpg')
    train_img.append(img_to_array(img))

x_train = np.array(train_img)

for i in range(n_test_data):
    img = load_img('uw-cs480-fall20/suffled-images/shuffled-images/' + str(test_id[i]) +'.jpg')
    test_img.append(img_to_array(img))
    
x_test = np.array(test_img)